## <span style="color:grey">Initialization</span>

In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import datetime
import logging
import copy 
import os
from multiprocessing import Process
warnings.filterwarnings('ignore')

exchange = 'NSE'
symbol = "HDFC"

today = datetime.datetime.now()
#today = datetime.datetime(2019, 4, 8, 15, 45, 0)

#configuration for downloading data
days =  120
years = 5
toDate = today
fromDate = toDate - datetime.timedelta(days = days)
freq = '1min'

mode = "Live" # Backtesting/Live
offline = False
if mode =="Live":
    offline = False

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"

from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0

%run "KiteConnect_Library.ipynb"    

### Kite - OAuth Login

In [2]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

logging.basicConfig(filename="log/live_log.log", filemode="a", level=logging.DEBUG, format="%(asctime)s;%(levelname)s;%(message)s")
logger=logging.getLogger()

kite = KiteConnect(api_key=KiteAPIKey)


f = open("kite_data/access_token.txt", mode="r")
access_token = f.readlines()
logger.info(access_token[0])
kite.set_access_token(access_token[0])

try:
    logger.info("Welcome "+kite.profile()['user_name'])
except:
    if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")
    driver.implicitly_wait(60)
    driver.get(kite.login_url())
    request_token = re.findall("request_token=(.*)",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    access_token = data["access_token"]
    kite.set_access_token(access_token)
    f = open("kite_data/access_token.txt", mode="w")
    f.write(access_token)
    f.close()

### Download NSE Stock Master List

In [3]:
if offline != True:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange, mode='w')

instruments_df = pd.read_hdf('kite_data/kite_data.h5', key=exchange, mode='r')

EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

### Filter Stocks - portfolio maker

In [4]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

downloadlist = nifty50['Symbol']
industry = niftynext50['Industry'].unique()

In [5]:
portfolio = pd.DataFrame(downloadlist)

In [6]:
portfolio = pd.DataFrame(['TCS', 'RIIL', 'HDFC','NDTV'])

portfolio = pd.DataFrame(['BAJAJ-AUTO','CIPLA','HINDUNILVR','M&M','WIPRO'])


#portfolio = pd.DataFrame(['BAJAJ-AUTO','CIPLA'])

#portfolio = pd.DataFrame(['BAJAJ-AUTO'])
portfolioToken = portfolio[0].apply(EQTOKEN)

### Download Historical Data - Equity

In [7]:
def portfolioDownload(stocklist, toDate):
    stocklist_df = pd.DataFrame()
    for index, row in stocklist.iterrows():
        symbol = row[0]
        logging.info("Downloading data for: "+symbol)
        temp_data = downloadData(symbol,  toDate - dt.timedelta(days = 1), toDate)
        temp_data['symbol'] = symbol
        temp_data.set_index(['symbol',temp_data.index], inplace=True)
        #print(temp_data)
        stocklist_df = stocklist_df.append(temp_data)
    
    #print(stocklist_df)
    return stocklist_df    

if offline != True:
    portfolio_df = pd.DataFrame()
    portfolio_df = portfolioDownload(portfolio, today)
    portfolio_df.to_hdf('kite_data/kite_data.h5', key="portfolio",mode='a')

#portfolio_df.loc['HDFC']

### Batch Downloader

In [8]:
def batchDownload(downloadlist):
    for symbol in downloadlist:
        logging.info("Downloading data for: "+symbol)

        raw_data = downloadData(symbol, fromDate, toDate)
        raw_data_day = downloadData(symbol,  toDate - dt.timedelta(days = years * 365),toDate, freq="day")

        raw_data_day.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
        raw_data.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')

### Incremental Download - untested

In [9]:
def incrementalDownload(downloadlist):
    for symbol in downloadlist:
        logging.info("Downloading data for: "+symbol)
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol,mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate)
        tempData = tempData.append(raw_data)
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')
        
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate, freq="day")
        
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
    
    

### Historical Data storage strategy

* minute level data and day level ohlc data is stored in the hd5
* symbol name is used as a key in the hd5 file system

### Tick Data storage strategy
- Tick data should not be merged with the downloaded historical data
- At the start of the trading session, last 60 candles from the historical data is fetched and stored in the dataframe for holding live data
- Two DataFrames ares created for storing live data: 1 for storing timestamp and LTP, another for OHLC
- OHLC data is created by grouping data first based on stock symbol and then based on timestamp(seconds and miliseconds are ignored)
- Streaming data from all the stocks in portfolio are strored in a single table. During post-procession subset of the master table sliced based on symbol is used
- Streaming data is resampled for minute frequency every minute to convert it to OHLC data which is stored in another dataframe
- Symbol is used as a key for the streaming OHLC dataframe storage

### Load Data from disk

In [10]:
if mode=="Live":
    raw_data = pd.read_hdf('kite_data/kite_data.h5', key="portfolio",mode='r')
else:
    raw_data = pd.read_hdf('kite_data/kite_data.h5', key="portfolio",mode='r')

##  <span style="color:green">Trading Strategies </span>

In [11]:
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0

UP = 0
DOWN = 1

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a, days: ROC(a,days) >= 0.01
TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        AroonDown, AroonUp = AROON(HIGH, LOW, 25)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
    
    def long_bull_engulf_ema(self):
        return pd.DataFrame( (EMA(CLOSE, 9) < EMA(CLOSE, 21)) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100) ,columns =["buy"])
    
    def long_hammer(self):
        return pd.DataFrame(TREND_DOWN(CLOSE, 10) & (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) != 0), columns=["buy"])
    
    def long_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(A,B) ,columns=["buy"])
    
    def long_aroon(self):
        return pd.DataFrame((AroonUp > 50) & (AroonDown<50), columns=["buy"])
        
    
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    def short_bear_engulf(self):
        return pd.DataFrame(TREND_UP(CLOSE,10) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100),columns=["sell"])
    
    def short_hanging_man(self):
        return pd.DataFrame(
            TREND_UP(CLOSE, 10) &
            (CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100)
            , columns=["sell"])
    
    def short_bear_kicking(self):
        return pd.DataFrame(CDLKICKING(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
        
    def short_shooting_star(self):
        return pd.DataFrame( TREND_UP(CLOSE, 5) & CDLSHOOTINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
    
    def short_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(B,A) ,columns=["sell"])
    
    def short_aroon(self):
        return pd.DataFrame((AroonUp < 50) & (AroonDown > 50), columns=["sell"])
    
    def tradeDecision(self):
        buy  = self.long_bull_engulf_ema() | self.long_bb() | self.long_ichimoku()
        #buy = self.long_aroon()
        sell = self.short_bear_engulf() | self.short_bb() | self.short_hanging_man() |  self.short_ichimoku()
        #sell = self.short_aroon()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
    
    def tradeRecommendation(self):
        buy, sell = self.tradeDecision()
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        logging.debug(str(timeNow))
        logging.debug(buy.tail(3))
        logging.debug(sell.tail(3))
        if (delT1 == buy.index[-1]) | (delT2 == buy.index[-1]) :
            return "buy,"+str(timeNow) +","+str(buy.iloc[-1]['low'])+","+str(buy.iloc[-1]['close'])+"\n"
        elif (delT1 == sell.index[-1]) | (delT2 == sell.index[-1]) :  
            return "sell,"+str(timeNow) +","+str(sell.iloc[-1]['high'])+","+str(sell.iloc[-1]['close'])+"\n"

if False:
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    a.tradeRecommendation()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")


## <span style="color:blue">BackTesting</span>

In [12]:
logger.setLevel(logging.CRITICAL)

### Multithreaded

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
from multiprocessing import Pool
def threadedBatchAnalysis(x):
    
    fig = ""
    temp_data = downloadData(x, fromDate, toDate, "day")
    #logging.debug(temp_data)
        
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig = createPlot(x)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    temp_data = aroon(temp_data,5, True)
    obv(temp_data,2, True)

    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    #print(os.getcwd())
    plotData()

if __name__ == '__main__':
    p = Pool(5)
    p.map_async(threadedBatchAnalysis, portfolio[0])
    p.apply_async(threadedBatchAnalysis, ['SBIN'])
    #portfolio
    
#threadedBatchAnalysis()

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



### Backtesting - Single Threaded

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
def batchAnalysis(portfolio=portfolio):
    global fig, noofcandles
    noofcandles = 50
    
    for index, row in portfolio.iterrows():
        fig = ""
        symbol = row[0] #row.iloc[:,0]
        temp_data = downloadData(symbol, fromDate, toDate, "day")
        
        #temp_data['symbol'] = symbol
        #temp_data.set_index(['temp_data.index], inplace=True)
        logging.debug(temp_data)
        
        yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
        yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()
        
        xMin = temp_data.index[-1*noofcandles]
        xMax = temp_data.index[-1]
        fig = createPlot(symbol)
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['yaxis']['range'] = [xMin, xMax]

        #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
        temp_data = candlestick(temp_data,1,True)
        temp_data = bbands(temp_data,1, True)

        temp_data = macd(temp_data,3,True)
        temp_data = rsi(temp_data,4, True)
        temp_data = aroon(temp_data,5, True)
        obv(temp_data,2, True)
        
        a = algoTrade(temp_data)
        buy, sell = a.tradeDecision()
        buy_df = annotateBuySell(buy, "Buy")
        sell_df = annotateBuySell(sell, "Sell")
        
        plotData(symbol)
        #input("Press Enter to continue")
    

#batchAnalysis()

## <span style="color:red"> Live Trading</span>

### Order Management

In [15]:
def buy(symbol, price, stoploss,quantity=1): 
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                trigger_price=stoploss,
                                price=price,
                                variety=kite.VARIETY_REGULAR
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
        
def buy_bo(symbol, price, stoploss,quantity=1): 
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                trigger_price=stoploss,
                                price=price,
                                variety=kite.VARIETY_BO
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))


def sell(symbol, price, stoploss, quantity=1): 
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=quantity,
                            order_type=kite.ORDER_TYPE_SL,
                            product=kite.PRODUCT_MIS,
                            trigger_price=stoploss,
                            price=price,
                            variety=kite.VARIETY_REGULAR)
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))

def getOrders():    
    # Fetch all orders
    return pd.DataFrame(kite.orders())

def cancelOrder(orderId):
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=orderId, parent_order_id=None)    
    except Exception as e:
        logging.info("Order Cancellation failed: {}".format(e.message))

In [16]:
buy("WIPRO",281.8,279)

In [36]:
sell("WIPRO",283,285)

There is an order update
There is an order update


In [24]:
cancelOrder(190411002308767)

In [50]:
positions_df = pd.DataFrame(kite.positions()['net'])
positions_df

,average_price,buy_m2m,buy_price,buy_quantity,buy_value,close_price,day_buy_price,day_buy_quantity,day_buy_value,day_sell_price,...,product,quantity,realised,sell_m2m,sell_price,sell_quantity,sell_value,tradingsymbol,unrealised,value
0,281.8,281.8,281.8,1,281.8,0,281.8,1,281.8,0,...,MIS,1,0,0,0,0,0,WIPRO,-0.3,-281.8


In [55]:
orders_df = getOrders()
orders_df[orders_df['status']!='REJECTED'][['order_id','exchange','tradingsymbol','transaction_type','price','trigger_price', 'quantity', 'order_type', 'product', 'status']]


,order_id,exchange,tradingsymbol,transaction_type,price,trigger_price,quantity,order_type,product,status
1,190411002167256,NSE,WIPRO,SELL,285.0,290,1,LIMIT,MIS,CANCELLED
2,190411002137522,NSE,WIPRO,BUY,280.0,279,1,LIMIT,MIS,CANCELLED
3,190411002298701,NSE,WIPRO,BUY,280.0,279,1,LIMIT,MIS,CANCELLED
4,190411002308767,NSE,WIPRO,SELL,290.0,291,1,LIMIT,MIS,CANCELLED
5,190411002348201,NSE,WIPRO,BUY,281.8,279,1,LIMIT,MIS,COMPLETE
6,190411002378318,NSE,WIPRO,SELL,282.0,285,1,LIMIT,MIS,CANCELLED
7,190411002477951,NSE,WIPRO,SELL,0.0,0,1,MARKET,MIS,COMPLETE


In [52]:
kite.modify_order(variety=kite.VARIETY_REGULAR, 
                  order_id=190411002378318, quantity=1, price=282, 
                  order_type=kite.ORDER_TYPE_SL)



'190411002378318'

There is an order update
There is an order update
There is an order update
There is an order update
There is an order update
There is an order update
There is an order update


In [24]:
logger.setLevel(logging.INFO)

### WebSocket: Tick Handler and sampler

In [20]:
import datetime

def resample(ws, freq="1min"):
    F = open("kite_data/recommendation.csv","a") 
    
    logging.debug(str(ws.prevtimeStamp)+": In resampler function")
    
    if ws.LiveStream.empty:
        logging.debug(str(ws.prevtimeStamp)+": Empty dataframe, Exiting resampler")
        return
    
    
    #logging.debug(ws.cbufferSampling.head())
    
  
    LiveStream2 = ws.LiveStream.groupby(['symbol','date']).agg({'price':['first','max','min','last'], 'volume':['last']})
    LiveStream2.columns = LiveStream2.columns.droplevel()
    LiveStream2.columns = ['open', 'high','low','close', 'volume']

    for index, data in LiveStream2.groupby(level=0):
        #print(index)
        sampled = data.loc[index].resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'volume':{'volume':'last'}})
        sampled.columns = sampled.columns.droplevel()

        #print(sampled)
        sampled['symbol'] = index
        sampled.set_index(['symbol',sampled.index], inplace=True)

        ws.LiveStreamOHLC = ws.LiveStreamOHLC.append(sampled)

    ws.LiveStreamOHLC.to_csv("kite_data/livestreamohlc.csv", mode='a')

    #print(ws.LiveStreamOHLC.loc['HDFC'].tail(1))
    a = algoTrade(ws.LiveStreamOHLC.loc[portfolio.iloc[-1,0]])
    reco_df = a.tradeRecommendation()

    logging.debug(ws.LiveStreamOHLC.loc[portfolio.iloc[-1,0]].tail(30))
    
    try:
        F.write(reco_df)
    except:
        logging.critical("Not able to resample and analyse data")
    finally:
        F.close()
        



def ticksHandler(ws, ticks):
    timeStamp = dt.datetime.now().replace(second=0, microsecond=0)
    tick_df = pd.DataFrame(ticks)
    
    #logging.debug("Prev: "+str(ws.prevtimeStamp)+"\nNow: "+str(timeStamp))
    
    try:
        tick_df.loc[tick_df['timestamp'].isna(), 'timestamp'] = timeStamp
        tick_df = tick_df[['timestamp','instrument_token','last_price','volume']]
        tick_df.instrument_token = tick_df.instrument_token.apply(EQSYMBOL)
        tick_df.columns = ['date','symbol','price','volume']
        tick_df.set_index(['symbol','date'], inplace=True)
        
        timeStamp = tick_df.index[1][-1].to_pydatetime()
        #print(type(timeStamp))
        
        #timeStamp = tick_df.loc[portfolio.iloc[-1,0]].index[-1]
    except:
        logging.debug("Exception: ticksHandler")
        
    if( (timeStamp - ws.prevtimeStamp) >= datetime.timedelta(minutes=1)):
        #logging.debug(tick_df)
        ws.prevtimeStamp = timeStamp
        
        #logging.debug(ws.cbufferAccum)
        
        ws.cbufferSampling = ws.cbufferAccum.copy(deep=True) #Copy accumulated stream to sampling buffer
        del ws.cbufferAccum
        #gc.collect()
        ws.cbufferAccum = pd.DataFrame() #Reset accumulation buffer
        #logging.debug(ws.cbufferSampling)
        resample(ws)
    
    ws.cbufferAccum = ws.cbufferAccum.append(tick_df)
    
    ws.LiveStream = ws.LiveStream.append(tick_df)
    ws.LiveStream.to_csv("kite_data/livestream.csv", mode='a')

def orderNotification(ws,data):
    logger.debug(data)

def initTrade(ws):
    ws.prevtimeStamp = datetime.datetime.now() - dt.timedelta(minutes=10)
    toDate = datetime.datetime.now()
    ws.cbufferAccum = pd.DataFrame()
    ws.cbufferSampling = pd.DataFrame()
    ws.LiveStream = pd.DataFrame()
    ws.LiveStreamOHLC = pd.DataFrame()
    ws.LiveStreamOHLC = portfolioDownload(portfolio, toDate)
    

### WebSocket: Kite Integration

In [21]:
#!python
from kiteconnect import KiteTicker

#logging.basicConfig(level=logging.CRITICAL)
logger.setLevel(logging.DEBUG)


# Initialise
kws = KiteTicker(KiteAPIKey, kite.access_token)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    #logging.debug("Ticks: {}".format(ticks))
    ticksHandler(ws, ticks)


def on_connect(ws, response):
    initTrade(ws)
    
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe(portfolioToken.tolist())

    ws.subscribe([225537])
    
    # Set RELIANCE to tick in `full` mode.
    # MODE_LTP, MODE_QUOTE, or MODE_FULL

    ws.set_mode(ws.MODE_FULL, portfolioToken.tolist())
    ws.set_mode(ws.MODE_FULL, [225537]) 
    #ws.set_mode(ws.MODE_LTP, [225537, 3861249]) 
    #ws.set_mode(ws.MODE_MODE_QUOTE, [2714625,779521]) 

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

def on_order_update(ws, data):
    logger.info("New Order Update")
    orderNotification(ws,data)

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_order_update = on_order_update

#kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.

### WebSocket: Start/Stop

In [22]:
kws.connect(threaded=True)

In [56]:
kws.close()

ERROR:kiteconnect.ticker:Connection closed: None - None


In [23]:
kws.is_connected()

True